In [1]:
# fashion_mnist_gui.py
import sys
import numpy as np
from PIL import Image
import tensorflow as tf
from PyQt6.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QFileDialog
from PyQt6.QtCore import Qt, QSize
from PyQt6.QtGui import QPixmap, QDragEnterEvent, QDropEvent

class FashionMNISTClassifier(QWidget):
    def __init__(self):
        super().__init__()
        # Class names for Fashion MNIST
        self.class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
        
        # Load TFLite model
        self.interpreter = tf.lite.Interpreter(model_path='fashion_mnist.tflite')
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Fashion MNIST Classifier')
        self.setAcceptDrops(True)
        self.setMinimumSize(QSize(400, 400))

        layout = QVBoxLayout()
        
        # Image display
        self.image_label = QLabel("Drag & Drop Image Here")
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.image_label.setStyleSheet("""
            QLabel {
                border: 2px dashed #aaa;
                padding: 20px;
                background-color: #f0f0f0;
            }
        """)

        # Prediction label
        self.prediction_label = QLabel("Prediction: ")
        self.prediction_label.setStyleSheet("font-size: 18px; font-weight: bold;")
        self.prediction_label.setAlignment(Qt.AlignmentFlag.AlignCenter)

        layout.addWidget(self.image_label)
        layout.addWidget(self.prediction_label)
        self.setLayout(layout)

    def dragEnterEvent(self, event: QDragEnterEvent):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event: QDropEvent):
        file_path = event.mimeData().urls()[0].toLocalFile()
        self.process_image(file_path)

    def process_image(self, file_path):
        try:
            """# Preprocess image
            img = Image.open(file_path).convert('L').resize((28, 28))
            img_array = np.array(img).reshape(1, 28, 28, 1).astype('float32') / 255.0"""

            # Preprocess image
            img = Image.open(file_path).convert('L').resize((28, 28))  # Convert to grayscale and resize
            img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize pixel values
            img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dims

            # Run inference
            self.interpreter.set_tensor(self.input_details[0]['index'], img_array)
            self.interpreter.invoke()
            predictions = self.interpreter.get_tensor(self.output_details[0]['index'])
            
            # Display results
            self.image_label.setPixmap(QPixmap(file_path).scaled(200, 200))
            predicted_class = self.class_names[np.argmax(predictions)]
            confidence = np.max(tf.nn.softmax(predictions))
            self.prediction_label.setText(
                f"Prediction: {predicted_class}\nConfidence: {confidence:.2%}"
            )

        except Exception as e:
            self.prediction_label.setText(f"Error: {str(e)}")


In [2]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Preprocess data
#x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
#x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy,#(from_logits=True),
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Convert to TFLite with quantization (OPTIONAL)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantization
tflite_quant_model = converter.convert()

# Save models
with open('fashion_mnist.tflite', 'wb') as f:
    f.write(tflite_quant_model)

# Save model summary (for slides)
with open('model_summary.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7547 - loss: 0.6761 - val_accuracy: 0.8667 - val_loss: 0.3632
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.8844 - loss: 0.3222 - val_accuracy: 0.8738 - val_loss: 0.3474
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9001 - loss: 0.2711 - val_accuracy: 0.8966 - val_loss: 0.2808
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9111 - loss: 0.2383 - val_accuracy: 0.8980 - val_loss: 0.2812
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9196 - loss: 0.2145 - val_accuracy: 0.9017 - val_loss: 0.2675
INFO:tensorflow:Assets written to: /var/folders/d5/_b51qwzx4f7__pl7x_l5jnkw0000gn/T/tmpzeibqlo1/assets


INFO:tensorflow:Assets written to: /var/folders/d5/_b51qwzx4f7__pl7x_l5jnkw0000gn/T/tmpzeibqlo1/assets


Saved artifact at '/var/folders/d5/_b51qwzx4f7__pl7x_l5jnkw0000gn/T/tmpzeibqlo1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  4423065744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423066896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423063440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423064976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423066320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423065936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423065168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423065552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423067664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4423068816: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1739130393.034831 5988148 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1739130393.034850 5988148 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-09 20:46:33.035252: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d5/_b51qwzx4f7__pl7x_l5jnkw0000gn/T/tmpzeibqlo1
2025-02-09 20:46:33.035694: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-09 20:46:33.035699: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d5/_b51qwzx4f7__pl7x_l5jnkw0000gn/T/tmpzeibqlo1
I0000 00:00:1739130393.039864 5988148 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-02-09 20:46:33.040625: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-02-09 20:46:33.066719: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder

In [3]:
app = QApplication(sys.argv)
window = FashionMNISTClassifier()
window.resize(400, 400)
window.show()
sys.exit(app.exec())

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2025-02-09 20:46:33.477 python[94799:5988148] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-09 20:46:33.477 python[94799:5988148] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/rohitpotdukhe/Documents/UniDocs/DSSS 2025/DSSS_Assignments_New/.conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
